# Obtenir els èxits de cada cançó del MSD

A continuació cercarem les cançons que coincideixen en clau (nom_canço, nom_autor) entre els datasets msd_reduced, billboard_reduced i grammy_reduced. Aquestes operacions es realitzaran mitjançant left joins entre el msd i els altres tres. Un cop localitzem les files coincidents, els hi assignarem un 1 i amb elles hi aplicarem les funcions exit_billboard(), exit_grammy() i exit_spotify() per determinar el valor del seu èxit.

Comencem definint una sèrie de funcions bàsiques que ens serviran al llarg del notebook.

In [1]:
import pandas as pd

In [2]:
# Retorna dataFrame a partir del csv filename

def get_songs_df_from_csv(filename):
    return pd.read_csv(filename,index_col=0)   

In [3]:
# Elimina les columnes del dataFrame passades per param

def drop_specific_cols(df, cols):
    df = df.drop(cols,axis=1)
    return df

Convertim els csv dels datasets grammy, msd, billboard i spotify en dataFrames.

In [4]:
# csv to df

grammy_df = get_songs_df_from_csv('./datasets/grammy_reduced.csv')
msd_df = get_songs_df_from_csv('./datasets/msd_reduced.csv')
billboard_df = get_songs_df_from_csv('./datasets/billboard_reduced.csv')

Realitzem els left joins de msd en grammy i billboard, prenent per clau el títol i nom de l'artista de cada cançó.

In [5]:
# Coincidències entre el million songs dataset i els datasets grammy, billboard i spotify

new_df = pd.merge(msd_df, grammy_df, how='left', left_on=['title','artist_name'], right_on = ['nominee','artist'])
new_df = pd.merge(new_df, billboard_df, how='left', left_on=['title','artist_name'], right_on = ['song','artist'])

In [6]:
display(new_df)

,danceability,duration,end_of_fade_in,energy,key,loudness,mode,start_of_fade_out,tempo,time_signature,...,title,category,nominee,artist_x,workers,rank,song,artist_y,peak-rank,weeks-on-board
0,0.0,252.05506,2.049,0.0,10,-4.829,0,236.635,87.002,4,...,Silent Night,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,156.55138,0.258,0.0,9,-10.555,1,148.660,150.778,1,...,Tanssi vaan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,138.97098,0.000,0.0,7,-2.060,1,138.971,177.768,4,...,No One Could Ever,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,145.05751,0.000,0.0,7,-4.654,1,138.687,87.433,4,...,Si Vos Querés,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,514.29832,0.000,0.0,5,-7.806,0,506.717,140.035,4,...,Tangle Of Aspens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926127,0.0,185.33832,0.000,0.0,1,-5.968,0,170.974,120.009,4,...,Down Fo' The Kick Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926128,0.0,217.44281,0.000,0.0,7,-8.339,0,217.443,92.159,7,...,O Samba Da Vida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926129,0.0,244.16608,3.048,0.0,5,-12.646,1,240.089,156.132,3,...,Jago Chhadeo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926130,0.0,553.03791,0.223,0.0,11,-8.218,0,548.989,137.089,4,...,Novemba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creem una nova columna, e_grammy, corresponent a l'èxit de la cançó pel que respecta als premis Grammy. Aquesta pren el valor 0 per a cançons que no apareixen als Grammy i n per a aquelles cançons que han rebut n Grammy's.

In [7]:
# Creem una nova columna, e_grammy, que pren valors binaris 0 o 1 segons si la cançó ha guanyat un Grammy.
# Ojo perquè una cançó pot guanyar més d'un Grammy.
new_df['e_grammy'] = new_df['nominee'].notnull().astype('int')

# Solució: sumar aquelles files que tinguin mateixa key (title, artist_name)
new_df.groupby(['title', 'artist_name'])['e_grammy'].sum().reset_index()

,title,artist_name,e_grammy
0,\tLatch,Valique,0
1,Cos My Babys Gone,The Kirkbys,0
2,Ia Oe E Ka La  For You_ O Glittering Sun ...,R. Carlos Nakai,0
3,N Ewigheid,Hi-5,0
4,Nuff Said,Megabusive,0
...,...,...,...
926087,ça Me Fait Du Bien,Emmanuel Moire,0
926088,étienne d'août,Malajube,0
926089,île d'yeu,Thomas Battenstein,0
926090,öffne dein herz,Turbolenz,0


In [8]:
new_df.describe()

,danceability,duration,end_of_fade_in,energy,key,loudness,mode,start_of_fade_out,tempo,time_signature,rank,peak-rank,weeks-on-board,e_grammy
count,926132.0,926132.000000,926132.000000,926132.0,926132.000000,926132.000000,926132.000000,926132.000000,926132.000000,926132.000000,7178.000000,7178.000000,7178.000000,926132.000000
mean,0.0,247.316303,0.821369,0.0,5.310618,-10.166678,0.668075,238.423916,123.826934,3.591480,75.088743,37.194344,13.253692,0.000491
std,0.0,125.824617,3.380694,0.0,3.595675,5.242923,0.470904,123.646934,35.253118,1.227213,23.150405,29.710305,7.758090,0.022160
min,0.0,0.313020,0.000000,0.0,0.000000,-58.178000,0.000000,0.313000,0.000000,0.000000,9.000000,1.000000,1.000000,0.000000
25%,0.0,180.009340,0.000000,0.0,2.000000,-12.744000,0.000000,172.426000,97.744000,3.000000,53.000000,10.000000,7.000000,0.000000
50%,0.0,227.878730,0.194000,0.0,5.000000,-8.998000,1.000000,218.813000,121.738000,4.000000,84.000000,31.000000,13.000000,0.000000
75%,0.0,286.745670,0.432000,0.0,9.000000,-6.387000,1.000000,276.468000,144.826250,4.000000,96.000000,61.000000,19.000000,0.000000
max,0.0,3034.905670,1163.941000,0.0,11.000000,4.318000,1.000000,3030.622000,302.300000,7.000000,100.000000,100.000000,69.000000,1.000000


Fem el mateix amb la billboard

In [9]:
new_df['is_billboard'] = new_df['peak-rank'].notnull().astype('int')

Però definim una mesura d'èxit no-binària

In [10]:
def billboard_success(m, s, is_billboard):
    if is_billboard:
        return (101 - m + s)
    else:
        return 0

In [11]:
# Per cada fila de la taula new_df, cridem la funció billboard_success amb els atributs seg. i guardem el valor computat a e_billboard
new_df['e_billboard'] = new_df.apply(lambda x: billboard_success(x['peak-rank'], x['weeks-on-board'],x['is_billboard']), axis=1)

In [12]:
display(new_df)

,danceability,duration,end_of_fade_in,energy,key,loudness,mode,start_of_fade_out,tempo,time_signature,...,artist_x,workers,rank,song,artist_y,peak-rank,weeks-on-board,e_grammy,is_billboard,e_billboard
0,0.0,252.05506,2.049,0.0,10,-4.829,0,236.635,87.002,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
1,0.0,156.55138,0.258,0.0,9,-10.555,1,148.660,150.778,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
2,0.0,138.97098,0.000,0.0,7,-2.060,1,138.971,177.768,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
3,0.0,145.05751,0.000,0.0,7,-4.654,1,138.687,87.433,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
4,0.0,514.29832,0.000,0.0,5,-7.806,0,506.717,140.035,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926127,0.0,185.33832,0.000,0.0,1,-5.968,0,170.974,120.009,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
926128,0.0,217.44281,0.000,0.0,7,-8.339,0,217.443,92.159,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
926129,0.0,244.16608,3.048,0.0,5,-12.646,1,240.089,156.132,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0
926130,0.0,553.03791,0.223,0.0,11,-8.218,0,548.989,137.089,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0


In [13]:
new_df.describe()

,danceability,duration,end_of_fade_in,energy,key,loudness,mode,start_of_fade_out,tempo,time_signature,rank,peak-rank,weeks-on-board,e_grammy,is_billboard,e_billboard
count,926132.0,926132.000000,926132.000000,926132.0,926132.000000,926132.000000,926132.000000,926132.000000,926132.000000,926132.000000,7178.000000,7178.000000,7178.000000,926132.000000,926132.000000,926132.000000
mean,0.0,247.316303,0.821369,0.0,5.310618,-10.166678,0.668075,238.423916,123.826934,3.591480,75.088743,37.194344,13.253692,0.000491,0.007751,0.597250
std,0.0,125.824617,3.380694,0.0,3.595675,5.242923,0.470904,123.646934,35.253118,1.227213,23.150405,29.710305,7.758090,0.022160,0.087695,7.446106
min,0.0,0.313020,0.000000,0.0,0.000000,-58.178000,0.000000,0.313000,0.000000,0.000000,9.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,0.0,180.009340,0.000000,0.0,2.000000,-12.744000,0.000000,172.426000,97.744000,3.000000,53.000000,10.000000,7.000000,0.000000,0.000000,0.000000
50%,0.0,227.878730,0.194000,0.0,5.000000,-8.998000,1.000000,218.813000,121.738000,4.000000,84.000000,31.000000,13.000000,0.000000,0.000000,0.000000
75%,0.0,286.745670,0.432000,0.0,9.000000,-6.387000,1.000000,276.468000,144.826250,4.000000,96.000000,61.000000,19.000000,0.000000,0.000000,0.000000
max,0.0,3034.905670,1163.941000,0.0,11.000000,4.318000,1.000000,3030.622000,302.300000,7.000000,100.000000,100.000000,69.000000,1.000000,1.000000,168.000000


In [14]:
# Eliminem columnes innecessàries per a estudiar les dades
cols = ['category', 'nominee', 'artist_x', 'workers', 'rank', 'song', 'artist_y', 'peak-rank', 'weeks-on-board', 'is_billboard']
new_df = drop_specific_cols(new_df, cols)

In [15]:
display(new_df)

,danceability,duration,end_of_fade_in,energy,key,loudness,mode,start_of_fade_out,tempo,time_signature,artist_name,title,e_grammy,e_billboard
0,0.0,252.05506,2.049,0.0,10,-4.829,0,236.635,87.002,4,Faster Pussy cat,Silent Night,0,0.0
1,0.0,156.55138,0.258,0.0,9,-10.555,1,148.660,150.778,1,Karkkiautomaatti,Tanssi vaan,0,0.0
2,0.0,138.97098,0.000,0.0,7,-2.060,1,138.971,177.768,4,Hudson Mohawke,No One Could Ever,0,0.0
3,0.0,145.05751,0.000,0.0,7,-4.654,1,138.687,87.433,4,Yerba Brava,Si Vos Querés,0,0.0
4,0.0,514.29832,0.000,0.0,5,-7.806,0,506.717,140.035,4,Der Mystic,Tangle Of Aspens,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926127,0.0,185.33832,0.000,0.0,1,-5.968,0,170.974,120.009,4,Killer Mike feat. Gangsta Pill and Nario of Gr...,Down Fo' The Kick Doe,0,0.0
926128,0.0,217.44281,0.000,0.0,7,-8.339,0,217.443,92.159,7,Kiko Navarro,O Samba Da Vida,0,0.0
926129,0.0,244.16608,3.048,0.0,5,-12.646,1,240.089,156.132,3,Kuldeep Manak,Jago Chhadeo,0,0.0
926130,0.0,553.03791,0.223,0.0,11,-8.218,0,548.989,137.089,4,Gabriel Le Mar,Novemba,0,0.0


In [16]:
# export to csv file
new_df.to_csv("successes.csv")